In [1]:
import pandas as pd
import re
import sys
import os

# Añadir la ruta del directorio data_utils al sys.path
# Agregar la carpeta raíz 'data_utils' al sys.path
sys.path.append(os.path.abspath('../'))


In [3]:
from data_utils.manipulacion_de_datos.limpieza_y_maniuplacion_de_datos import  procesar_descripcion
from data_utils.funciones_de_busqueda.busqueda_por_filtro import filtrar_por_laboratorio
from data_utils.funciones_de_busqueda.busquedas_por_texto import main_principal

Carga de datos CSV 

In [5]:
#Carga de datos CSV 

#Carga de Producto
df_Producto = pd.read_csv("L_STMaestroPrd.csv")
# Eliminar filas donde todas sus columnas están vacías
df_MasterProductos = df_Producto.dropna(how='all')

# Eliminar columnas donde todos sus valores están vacíos
df_MasterProductos = df_MasterProductos.dropna(axis=1, how='all')
df_MasterProductos["descripcion_limpia_producto"] = df_MasterProductos["descrip1"].fillna('') + " " + df_MasterProductos["descrip2"].fillna('')

#Carga Proveedor
df_Proveedor = pd.read_csv('Phoenix.csv')
# Eliminar filas donde todas sus columnas están vacías
df_Proveedor = df_Proveedor.dropna(how='all')

# Eliminar columnas donde todos sus valores están vacíos
df_Proveedor = df_Proveedor.dropna(axis=1, how='all')

df_Proveedor['descripcion_limpia_proveedor'] = df_Proveedor['descripcion']

C:\Users\Windows\AppData\Local\Temp\ipykernel_7624\2265950528.py:4: DtypeWarning: Columns (7,14,17,18,21,28,39,42,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df_Producto = pd.read_csv("L_STMaestroPrd.csv")


Estandarizacion de COLUMNAS

In [7]:
# Procesamiento de df_MasterProductos: descripcion_limpia_producto
df_MasterProductos = procesar_descripcion(df_MasterProductos, "descripcion_limpia_producto")
# Procesamiento de df_Proveedor: descripcion_limpia_proveedor
df_Proveedor = procesar_descripcion(df_Proveedor, 'descripcion_limpia_proveedor')

In [8]:
# Pedir al usuario que ingrese el nombre del laboratorio
laboratorio_input = input("Ingresa el nombre del laboratorio: ")  # El usuario ingresa el nombre del laboratorio
df_MasterProductos_Filtro = filtrar_por_laboratorio(df_MasterProductos, laboratorio_input)

Se encontraron 402 productos coincidentes del laboratorio 'Phoenix'.


Procesamiento para comparacion de  Descripciones

In [27]:
#Resultado de comparacion entre ambas descripciones 
df_resultado_similiridad = main_principal(df_MasterProductos_Filtro, df_Proveedor)
df_resultado_similiridad.columns

Index(['similaridad', 'publicado/\r\nno publicado_x', 'u n_x', 'linea_x',
       'ean_x', 'cod. dispro_x', 'cod. elea_x', 'cod. cid latina_x',
       'registro alfabeta_x', 'codigo kairos_x', 'pami_x', 'certif nro_x',
       'descripcion_x', 'presentacion_x', 'descripcion_limpia_proveedor',
       'niprod', 'cod_prod', 'cod_abrev', 'descrip1', 'descrip2', 'abrev',
       'cod_catprod', 'codbarra', 'cod_fbarra', 'unid_v', 'base_v', 'precio',
       'mon_v', 'costo', 'base_c', 'unid_c', 'mon_c', 'rel_c_v', 'rel_v_c',
       'unid_alt', 'rel_c_a', 'rel_a_c', 'cod_mone', 'habilitado', 'cod_bonif',
       'pped', 'st_min', 'st_max', 'observ', 'pc_comision', 'pc_utilidad',
       'ind_stock', 'ind_partidas', 'ind_camestado', 'ind_series', 'ind_fvto',
       'ninum', 'tipo_egreso', 'pc_diftrans', 'cod_cvta', 'cat_item',
       'cant_rep', 'pto_rep', 'atrib0', 'atrib1', 'atrib2',
       'descripcion_limpia_producto', 'publicado/\r\nno publicado_y', 'u n_y',
       'linea_y', 'ean_y', 'cod. dis

In [25]:
df_resultado_similiridad[df_resultado_similiridad['similaridad'] >= 0.79][['similaridad', 'niprod',"ean_x", 'descripcion_limpia_proveedor', 'descripcion_limpia_producto', 'atrib0']]

,similaridad,niprod,ean_x,descripcion_limpia_proveedor,descripcion_limpia_producto,atrib0
470,0.938959,91093.0,7791848079022,loreme x descongestivo comprimidos x 5,loreme x descongestivo comprimidos x 10,PHOENIX
492,0.796588,91147.0,7791848282026,nefazan 75 miligramos x 30 comprimidos rec,nefazan 75 miligramos comprimidos x 60,PHOENIX
513,0.890747,91160.0,7791848245007,neumote x nebu x 20 mililitros,neumote x nebu envase x 20 mililitros,PHOENIX


In [19]:
# Buscar el valor 102876 en la columna 'niprod'
#resultado = df_MasterProductos_Filtro[df_MasterProductos_Filtro['niprod'] == 16787]

resultado = df_MasterProductos[df_MasterProductos['niprod'] == 91093]
resultado
# Ver el resultado
#resultado['descripcion_limpia_producto'].tolist()



,niprod,cod_prod,cod_abrev,descrip1,descrip2,abrev,cod_catprod,codbarra,cod_fbarra,unid_v,...,tipo_egreso,pc_diftrans,cod_cvta,cat_item,cant_rep,pto_rep,atrib0,atrib1,atrib2,descripcion_limpia_producto
32662,91093.0,20310,20310,LOREMEX DESCONGESTIVO comp.x 10,NaN,LOREMEX DESCONGESTIV,PT,7791848079015,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,loreme x descongestivo comprimidos x 10


Prueba de  NUEVO ENFOQUE: HIBRIDO


In [20]:

def procesar_descripcion_HIBRIDO(data, nombre_columna):
    """
    Procesa una columna de un DataFrame aplicando normalización y manejo de combinaciones de texto.
    
    Parámetros:
    - data: DataFrame que contiene la columna a procesar.
    - nombre_columna: Nombre de la columna que se va a procesar.
    
    Retorna:
    - DataFrame con la columna procesada y actualizada.
    """
    # Verificar si la columna existe en el DataFrame
    if nombre_columna not in data.columns:
        raise KeyError(f"La columna '{nombre_columna}' no existe en el DataFrame.")
    
    # Función para normalizar el texto
    def normalizar(texto):
        # Asegurar que el texto sea un string
        texto = str(texto).strip().lower()
        
        # 1. Eliminar espacios redundantes
        texto = re.sub(r'\s+', ' ', texto)

        # 2. Remover caracteres especiales excepto letras, números, espacios y '%'
        texto = re.sub(r'[^a-zA-Z0-9\s%]', '', texto)

        # 3. Separar números de letras y letras de números
        texto = re.sub(r'(\d)([a-zA-Z])', r'\1 \2', texto)
        texto = re.sub(r'([a-zA-Z])(\d)', r'\1 \2', texto)

        # 4. Manejo de 'x':
        texto = re.sub(r'(?<=\d)x(?=\d)', r' x ', texto)  # separar x entre números
        texto = re.sub(r'\b([a-zA-Z]+)x\b', r'\1 x', texto)  # separar palabras terminadas en x

        # 5. Reemplazar separadores como guiones o barras con espacios
        texto = re.sub(r'[-/]', ' ', texto)

        # 6. Manejo de unidades (por ejemplo, separar g, ml, etc. de los números)
        texto = re.sub(r'(\d+)\s*([a-zA-Z]+)', r'\1 \2', texto)

        # 7. Eliminar espacios redundantes nuevamente
        texto = texto.strip()

        # 8. Combinaciones
        combinaciones = {
            'anemido x': 'anemidox',
            'jgaprell' : 'jga prell',
            'fle x': 'flex',
            'argeflo x':'argeflox',
            'aropa x': 'aropax',
            'arolte x' : 'aroltex',
            'atopi x':  'atopix',
            'atorma x': 'atormax',
            'comprec': 'comp recubierto',
            'soloft': 'solucion oftalmica',
            'g': 'gr',
            'cr': 'crema',
            'comp' : 'comprimidos',
            'caps': 'capsulas',
            "jer.pre" : "jeringa prell",
            "mg" : "miligramos",
            "ml" : "mililitros",
            "mm" : "milimetros",
            'zyvo x': 'zyvox',
            'loc' : 'locion',
            "gr" : "gramos",
            "jga" : 'jeringa',
            "cm" : 'centimetros',
            'u' : "unidad",
            'shamp' : 'shampoo',
            "unid" : " unidad",
            'cep' : 'cepillo dental',
            "emuls prot" : 'emulsion protectora',
            'emulshidrat':'emulsion hidratante',
            'emulshumect' : "emulsion humectante",
            'iny': 'inyeccion',
            'amp' : 'ampolla',
            'inylioffa': 'inyeccion lioffa',
            'ds' : 'dosis',
            'inyfa': 'inyeccion fa',
            'jbe': 'jarabe',
            'blist': 'blister',
            'sob' : 'sobres',
            'fa': 'frasco ampolla',
            'jab' : 'jabon',
            'cps' :'capsulas',
            'ivi x' : 'ivix',
            'inya': 'inyectable',
            'inyjgaprell' : 'iny jeringa prellenada',
            'mcg': 'microgramo',
            'gts': 'gotas',
            'mgml' : 'miligramos por mililitro',
            'env': 'envase',
            'gtsoft' : 'gotas oftálmicas',
            'compdisp': 'comprimidos disp',
            'comprecran' : 'comprimidos recubiertos ranurados',
            'sol': 'solucion',
            'jgapre': 'jeringa prellenada',
            'fcocps' : 'frasco capsulas',
            'fcogotero' : 'frasco gotero',
            'fco': 'frasco', 
            'solspray' : 'solucion spray',
            'solpuas' : 'sol puas',
            'cpsblandas' : 'cps blandas',
            'tab' : 'tableta',
            'sach' : 'sachet',
            'mgvial': 'miligramos vial',
            'cpsbl' :'capsula blanda',
            "recub" :"recubiertos",
        }
        for key, val in combinaciones.items():
            texto = re.sub(rf'\b{key}\b', val, texto)

        # Convertir números a float (o int) para que se traten como valores numéricos
        def convertir_a_numero(palabra):
            try:
                return float(palabra) if '.' in palabra else int(palabra)
            except ValueError:
                return palabra
        
        # Aplicar la conversión a cada palabra
        texto = " ".join([str(convertir_a_numero(palabra)) for palabra in texto.split()])

        return texto

    # Aplicar la función 'normalizar' a cada elemento de la columna
    data[nombre_columna] = data[nombre_columna].apply(normalizar)
    
    # Aquí aplicamos el split
    data[nombre_columna] = data[nombre_columna].str.split(' ')

    return data  # Retorna el DataFrame con la columna procesada y la nueva columna de split


In [21]:
#Procesamiento de Columna Descripcion
# Procesamiento de df_MasterProductos: descripcion_limpia_producto

df_MasterProductos_Filtro_hibrido = procesar_descripcion_HIBRIDO(df_MasterProductos_Filtro, "descripcion_limpia_producto")

# Procesamiento de df_Proveedor: descripcion_limpia_proveedor
df_Proveedor_hibrido = procesar_descripcion_HIBRIDO(df_Proveedor, 'descripcion_limpia_proveedor')

df_MasterProductos_Filtro_hibrido

C:\Users\Windows\AppData\Local\Temp\ipykernel_13748\47486557.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[nombre_columna] = data[nombre_columna].apply(normalizar)
C:\Users\Windows\AppData\Local\Temp\ipykernel_13748\47486557.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[nombre_columna] = data[nombre_columna].str.split(' ')


,niprod,cod_prod,cod_abrev,descrip1,descrip2,abrev,cod_catprod,codbarra,cod_fbarra,unid_v,...,tipo_egreso,pc_diftrans,cod_cvta,cat_item,cant_rep,pto_rep,atrib0,atrib1,atrib2,descripcion_limpia_producto
604,26458.0,33228,33228,ACOTRAL 10 mg comp.x 10,NaN,ACOTRAL 10 mg comp.x,PT,7791848286017,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,"[acotral, 10, miligramos, comprimidos, x, 10]"
605,26459.0,33229,33229,ACOTRAL 10 mg comp.x 20,NaN,ACOTRAL 10 mg comp.x,PT,7791848286024,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,"[acotral, 10, miligramos, comprimidos, x, 20]"
606,26460.0,33751,33751,ACOTRAL 10 mg comp.x 30,NaN,ACOTRAL 10 mg comp.x,PT,7791848286031,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,"[acotral, 10, miligramos, comprimidos, x, 30]"
1266,26461.0,33082,33082,AGIOFIBRAS gran.x 250 g,NaN,AGIOFIBRAS gran.x 25,PT,7791848259066,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,"[agiofibras, gran, x, 250, gramos]"
1267,26462.0,46479,46479,AGIOFIBRAS sob.x 12,NaN,AGIOFIBRAS sob.x 12,PT,7791848259097,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,"[agiofibras, sobres, x, 12]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,91314.0,23342,23342,XANOL 12 mcg caps.x 30,NaN,XANOL 12 mcg caps.x,PT,7791848262028.0,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,"[xanol, 12, microgramo, capsulas, x, 30]"
56038,91315.0,25874,25874,XANOL 12 mcg caps.x 60,NaN,XANOL 12 mcg caps.x,PT,7791848262042.0,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,"[xanol, 12, microgramo, capsulas, x, 60]"
56039,91316.0,23341,23341,XANOL 6 mcg caps.x 30,NaN,XANOL 6 mcg caps.x 3,PT,7791848262011.0,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,"[xanol, 6, microgramo, capsulas, x, 30]"
56040,91317.0,25873,25873,XANOL 6 mcg caps.x 60,NaN,XANOL 6 mcg caps.x 6,PT,7791848262035.0,NaN,UN,...,E,0.0,NaN,ITGEV,0,0.0,PHOENIX,NaN,NaN,"[xanol, 6, microgramo, capsulas, x, 60]"


In [22]:
import re

def separar_numeros_y_descripcion_lista(descripcion_lista):
    """
    Separa los números de las palabras en la lista de descripción.
    Devuelve dos listas: una con las palabras (sin números) y otra con los números.
    """
    numeros = []
    descripcion_sin_numeros = []

    for item in descripcion_lista:
        # Si el item es un número (verificamos con regex para soportar enteros y decimales)
        if re.match(r'^\d+(\.\d+)?$', item):  # Verificamos si es número
            numeros.append(item)
        else:
            descripcion_sin_numeros.append(item)
    
    return descripcion_sin_numeros, numeros

# Aplicar la función a las descripciones de los datasets
df_MasterProductos_Filtro_hibrido['descripcion_sin_numeros'], df_MasterProductos_Filtro_hibrido['numeros'] = zip(*df_MasterProductos_Filtro_hibrido['descripcion_limpia_producto'].apply(separar_numeros_y_descripcion_lista))
df_Proveedor_hibrido['descripcion_sin_numeros'], df_Proveedor_hibrido['numeros'] = zip(*df_Proveedor_hibrido['descripcion_limpia_proveedor'].apply(separar_numeros_y_descripcion_lista))


C:\Users\Windows\AppData\Local\Temp\ipykernel_13748\3147362355.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_MasterProductos_Filtro_hibrido['descripcion_sin_numeros'], df_MasterProductos_Filtro_hibrido['numeros'] = zip(*df_MasterProductos_Filtro_hibrido['descripcion_limpia_producto'].apply(separar_numeros_y_descripcion_lista))
C:\Users\Windows\AppData\Local\Temp\ipykernel_13748\3147362355.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_MasterProductos_Filtro_hibrido['descripcion_sin_numero

In [23]:
df_MasterProductos_Filtro_hibrido['descripcion_sin_numeros']

604      [acotral, miligramos, comprimidos, x]
605      [acotral, miligramos, comprimidos, x]
606      [acotral, miligramos, comprimidos, x]
1266             [agiofibras, gran, x, gramos]
1267                   [agiofibras, sobres, x]
                         ...                  
56037         [xanol, microgramo, capsulas, x]
56038         [xanol, microgramo, capsulas, x]
56039         [xanol, microgramo, capsulas, x]
56040         [xanol, microgramo, capsulas, x]
56118                    [xeomin, ui, vial, x]
Name: descripcion_sin_numeros, Length: 402, dtype: object

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Asegurarse de que las descripciones estén en formato de texto (cadena) y no como listas de palabras
df_MasterProductos_Filtro_hibrido['descripcion_sin_numeros'] = df_MasterProductos_Filtro_hibrido['descripcion_sin_numeros'].apply(lambda x: ' '.join(x))
df_Proveedor_hibrido['descripcion_sin_numeros'] = df_Proveedor_hibrido['descripcion_sin_numeros'].apply(lambda x: ' '.join(x))

# Crear el vectorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Crear matrices TF-IDF separadas para las descripciones de productos maestros y proveedores
tfidf_master = tfidf_vectorizer.fit_transform(df_MasterProductos_Filtro_hibrido['descripcion_sin_numeros'])
tfidf_proveedor = tfidf_vectorizer.transform(df_Proveedor_hibrido['descripcion_sin_numeros'])

# Calcular la similitud del coseno entre las descripciones de los proveedores y las de los productos maestros
cos_sim_matrix = cosine_similarity(tfidf_proveedor, tfidf_master)

# Mostrar la matriz de similitud
print(cos_sim_matrix)


[[0.08802714 0.08802714 0.08802714 ... 0.         0.         0.        ]
 [0.08802714 0.08802714 0.08802714 ... 0.         0.         0.        ]
 [0.08642029 0.08642029 0.08642029 ... 0.         0.         0.        ]
 ...
 [0.40960935 0.40960935 0.40960935 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


C:\Users\Windows\AppData\Local\Temp\ipykernel_13748\4232402614.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_MasterProductos_Filtro_hibrido['descripcion_sin_numeros'] = df_MasterProductos_Filtro_hibrido['descripcion_sin_numeros'].apply(lambda x: ' '.join(x))


In [25]:
cos_sim_matrix

array([[0.08802714, 0.08802714, 0.08802714, ..., 0.        , 0.        ,
        0.        ],
       [0.08802714, 0.08802714, 0.08802714, ..., 0.        , 0.        ,
        0.        ],
       [0.08642029, 0.08642029, 0.08642029, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.40960935, 0.40960935, 0.40960935, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], shape=(702, 402))

In [26]:
import numpy as np

def calcular_diferencia_numerica(numeros1, numeros2):
    """
    Calcula la diferencia relativa promedio entre dos listas de números.
    """
    # Convertir las listas de texto a números (float o int según sea necesario)
    numeros1 = np.array([float(num) for num in numeros1])
    numeros2 = np.array([float(num) for num in numeros2])

    if not len(numeros1) or not len(numeros2):  # Si alguna lista está vacía
        return 1  # Máxima diferencia si no hay números para comparar
    
    # Tomar la longitud mínima para comparar solo elementos coincidentes
    min_len = min(len(numeros1), len(numeros2))
    numeros1, numeros2 = numeros1[:min_len], numeros2[:min_len]
    
    # Calcular diferencia relativa promedio
    diferencia = np.abs(numeros1 - numeros2) / numeros1
    return np.mean(diferencia)

# Ejemplo de uso entre productos
diferencias_numericas = []
for idx_prov, numeros_prov in enumerate(df_Proveedor_hibrido['numeros']):
    diffs = [
        calcular_diferencia_numerica(numeros_prov, numeros_master)
        for numeros_master in df_MasterProductos_Filtro_hibrido['numeros']
    ]
    diferencias_numericas.append(diffs)


In [27]:
# Pesos para el cálculo híbrido
peso_texto = 0.3
peso_numeros = 0.7

# Calcular la similitud final para cada producto del proveedor
resultados = []
for idx_prov, similitudes_texto in enumerate(cos_sim_matrix):
    mejores_resultados = []
    for idx_master, sim_texto in enumerate(similitudes_texto):
        # Obtener la diferencia numérica correspondiente
        diferencia_num = diferencias_numericas[idx_prov][idx_master]
        # Calcular similitud numérica (1 - diferencia)
        sim_num = 1 - diferencia_num
        # Calcular similitud final ponderada
        sim_final = (peso_texto * sim_texto) + (peso_numeros * sim_num)
        mejores_resultados.append((idx_master, sim_final))
    # Seleccionar el mejor match por similitud final
    mejor_match = max(mejores_resultados, key=lambda x: x[1])
    resultados.append({
        "producto_proveedor": df_Proveedor_hibrido.iloc[idx_prov]['descripcion_limpia_proveedor'],
        "producto_master": df_MasterProductos_Filtro_hibrido.iloc[mejor_match[0]]['descripcion_limpia_producto'],
        "similitud_final": mejor_match[1]
    })

# Crear un DataFrame con los resultados
df_resultados = pd.DataFrame(resultados)


In [28]:
# Filtrar los resultados con similitud_final > 0.7
df_resultados

,producto_proveedor,producto_master,similitud_final
0,"[3, tc, azt, elea, envase, x, 60, comprimidos]","[doxolbran, 2, miligramos, comprimidos, x, 30]",0.434741
1,"[abacavir, elea, envase, x, 60, comprimidos, r...","[mosar, plus, comprimidos, x, 60]",0.724012
2,"[alcaf, 60, miligramos, f, ampolla, polvo, x, 1]","[colchicina, phoeni, x, compran, x, 60]",0.700000
3,"[benjor, 50, miligramos, x, comprimidos, x, 30]","[pioglit, 45, miligramos, comprimidos, x, 30]",0.798892
4,"[beva, x, 100, miligramos, envase, x, 1, vial,...","[xeomin, 100, ui, vial, x, 1]",0.802344
...,...,...,...
697,"[zatium, met, 50500, miligramos, comprimidos, ...","[neumotide, capsulas, 50050, microgramo, capsu...",0.696881
698,"[zatium, met, 50850, miligramos, comprimidos, ...","[pioglit, 45, miligramos, comprimidos, x, 30]",0.484201
699,"[zatium, met, 50850, miligramos, comprimidos, ...","[neumotide, capsulas, 50050, microgramo, capsu...",0.694494
700,"[zopirol, 050%, solucion, oftalmica, x, 5, mil...","[clare, x, compuesto, solucion, oftalmica, x, ...",0.915372
